# Apply posterior reweighting

mention in paper: skew-normal distribution

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scipy.stats
import scipy.optimize
from tqdm.auto import tqdm

from dotenv import dotenv_values
from fair import __version__

In [ ]:
cal_v = dotenv_values("../../../.env")["CALIBRATION_VERSION"]
samples = int(dotenv_values("../../../.env")["PRIOR_SAMPLES"])
output_ensemble_size = int(dotenv_values("../../../.env")["POSTERIOR_SAMPLES"])
fair_v = dotenv_values("../../../.env")["FAIR_VERSION"]
constraint_set = dotenv_values("../../../.env")["CONSTRAINT_SET"]

assert fair_v == __version__

In [ ]:
# # pl.rcParams['figure.figsize'] = (11.4, 11.4)
# pl.rcParams['font.size'] = 16
# pl.rcParams['font.family'] = 'Arial'
# pl.rcParams['ytick.direction'] = 'in'
# pl.rcParams['ytick.minor.visible'] = True
# pl.rcParams['ytick.major.right'] = True
# pl.rcParams['ytick.right'] = True
# pl.rcParams['xtick.direction'] = 'in'
# pl.rcParams['xtick.minor.visible'] = True
# pl.rcParams['xtick.major.top'] = True
# pl.rcParams['xtick.top'] = True
# pl.rcParams['axes.spines.top'] = True
# pl.rcParams['axes.spines.bottom'] = True
# pl.rcParams['figure.dpi'] = 150
# pl.rcParams['lines.linewidth'] = 2

In [ ]:
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)

In [ ]:
valid_temp = np.loadtxt(f'../../../output/fair-{fair_v}/v{cal_v}/posteriors/{constraint_set}/runids_rmse_pass.csv').astype(np.int64)

In [ ]:
input_ensemble_size = len(valid_temp)
input_ensemble_size

In [ ]:
assert input_ensemble_size > output_ensemble_size

In [ ]:
temp_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/temperature_1850-2101.npy')
ohc_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/ocean_heat_content_2018_minus_1971.npy')
fari_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/forcing_ari_2005-2014_mean.npy')
faci_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/forcing_aci_2005-2014_mean.npy')
co2_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/concentration_co2_2014.npy')
ecs_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/ecs.npy')
tcr_in = np.load(f'../../../output/fair-{fair_v}/v{cal_v}/prior_runs/tcr.npy')
faer_in = fari_in + faci_in

In [ ]:
def opt(x, q05_desired, q50_desired, q95_desired):
    "x is (a, loc, scale) in that order."
    q05, q50, q95 = scipy.stats.skewnorm.ppf((0.05, 0.50, 0.95), x[0], loc=x[1], scale=x[2])
    #print(q05, q50, q95, x)
    return (q05-q05_desired, q50-q50_desired, q95-q95_desired)
    
#scipy.optimize.root(opt, (2.0, 2.6, 1.2), (1.24, 1.81, 2.59), options={'maxfev': 5000})

In [ ]:
#scipy.optimize.root(opt, (1, 0.85, 1), (0.69, 0.85, 0.98), options={'maxfev': 5000})

In [ ]:
samples = {}
samples['ECS'] = scipy.stats.skewnorm.rvs(8.82185594, loc=1.95059779, scale=1.55584604, size=10**5, random_state=91603)
samples['TCR'] = scipy.stats.norm.rvs(loc=1.8, scale=0.6/NINETY_TO_ONESIGMA, size=10**5, random_state=18196)
samples['OHC'] = scipy.stats.norm.rvs(loc=434.3730268913012, scale=73.70562981598447, size=10**5, random_state=43178)
samples['temperature 1995-2014'] = scipy.stats.skewnorm.rvs(-1.65506091, loc=0.92708099, scale=0.12096636, size=10**5, random_state=19387)
samples['temperature 2081-2100'] = scipy.stats.skewnorm.rvs(2.20496701, loc=1.4124379, scale=0.60080822, size=10**5, random_state=91831)
samples['ERFari'] = scipy.stats.norm.rvs(loc=-0.3, scale=0.3/NINETY_TO_ONESIGMA, size=10**5, random_state=70173)
samples['ERFaci'] = scipy.stats.norm.rvs(loc=-1.0, scale=0.7/NINETY_TO_ONESIGMA, size=10**5, random_state=91123)
samples['ERFaer'] = scipy.stats.norm.rvs(loc=-1.3, scale=np.sqrt(0.7**2+0.3**2)/NINETY_TO_ONESIGMA, size=10**5, random_state=3916153)
samples['CO2 concentration'] = scipy.stats.norm.rvs(loc=397.5469792683919, scale=0.36, size=10**5, random_state=81693)

In [ ]:
ohc_in

In [ ]:
# pl.hist(samples['ECS'], bins=100, alpha=0.5, label='Target distribution', density=True)
# pl.hist(ecs_in[valid_temp], bins=100, alpha=0.5, label='RMSE < 0.16 K posterior', density=True)
# pl.legend()
# print('target distribution:', np.percentile(samples['ECS'], (5, 16, 50, 84, 95)))
# print('naive constraints:', np.percentile(ecs_in[valid_temp], (5, 16, 50, 84, 95)))
# pl.title('Equilibrium climate sensitivity')
# pl.xlim(0, 8)
# pl.ylabel('Density')
# pl.xlabel('deg C')
# # pl.savefig('../plots/reweighting.png')

In [ ]:
ar_distributions = {}
for constraint in ['ECS', 'TCR', 'OHC', 'temperature 1995-2014', 'temperature 2081-2100', 'ERFari', 'ERFaci', 'ERFaer', 'CO2 concentration']:
    ar_distributions[constraint] = {}
    ar_distributions[constraint]['bins'] = np.histogram(samples[constraint], bins=200, density=True)[1]
    ar_distributions[constraint]['values'] = samples[constraint]

In [ ]:
weights_20yr = np.ones(21)
weights_20yr[0] = 0.5
weights_20yr[-1] = 0.5
weights_51yr = np.ones(52)
weights_51yr[0] = 0.5
weights_51yr[-1] = 0.5

In [ ]:
accepted = pd.DataFrame(
    {
        'ECS': ecs_in[valid_temp],
        'TCR': tcr_in[valid_temp],
        'OHC': ohc_in[valid_temp]/1e21,
        'temperature 1995-2014': np.average(temp_in[145:166,valid_temp], weights=weights_20yr, axis=0) - np.average(temp_in[:52,valid_temp], weights=weights_51yr, axis=0),
        'temperature 2081-2100': np.average(temp_in[231:252,valid_temp], weights=weights_20yr, axis=0) - np.average(temp_in[145:166,valid_temp], weights=weights_20yr, axis=0),
        'ERFari': fari_in[valid_temp],
        'ERFaci': faci_in[valid_temp],
        'ERFaer': faer_in[valid_temp],
        'CO2 concentration': co2_in[valid_temp]
    },
    index=valid_temp
)

In [ ]:
accepted

In [ ]:
def calculate_sample_weights(distributions, samples, niterations=50):
    #weights = np.ones(samples[list(accepted.keys())[0]].shape[0])
    weights = np.ones(samples.shape[0])
    gofs = []
    gofs_full = []

    unique_codes = list(distributions.keys())#[::-1]

    for k in tqdm(range(niterations), desc="Iterations", leave=False):
        gofs.append([])
        if k == (niterations - 1):
            weights_second_last_iteration = weights.copy()
            weights_to_average = []
            
        for j, unique_code in enumerate(unique_codes):
            unique_code_weights, our_values_bin_idx = get_unique_code_weights(
                unique_code, distributions, samples, weights, j, k
            )
            if k == (niterations - 1):
                weights_to_average.append(unique_code_weights[our_values_bin_idx])
        
            weights *= unique_code_weights[our_values_bin_idx]
            
            gof = ((unique_code_weights[1:-1] - 1) ** 2).sum()
            gofs[-1].append(gof)
               
            gofs_full.append([unique_code])
            for unique_code_check in unique_codes:
                unique_code_check_weights, _ = get_unique_code_weights(
                    unique_code_check, distributions, samples, weights, 1, 1
                )
                gof = ((unique_code_check_weights[1:-1] - 1) ** 2).sum()
                gofs_full[-1].append(gof)

    weights_stacked = np.vstack(weights_to_average).mean(axis=0)
    weights_final = weights_stacked * weights_second_last_iteration

    gofs_full.append(["Final iteration"])
    for unique_code_check in unique_codes:
        unique_code_check_weights, _ = get_unique_code_weights(
            unique_code_check, distributions, samples, weights_final, 1, 1
        )
        gof = ((unique_code_check_weights[1:-1] - 1) ** 2).sum()
        gofs_full[-1].append(gof)

    return (
        weights_final, 
        pd.DataFrame(np.array(gofs), columns=unique_codes), 
        pd.DataFrame(np.array(gofs_full), columns=["Target marginal"] + unique_codes)
    )


In [ ]:
def get_unique_code_weights(unique_code, distributions, samples, weights, j, k):
    bin_edges = distributions[unique_code]["bins"]
    our_values = samples[unique_code].copy()

    our_values_bin_counts, bin_edges_np = np.histogram(
        our_values, bins=bin_edges
    )
    np.testing.assert_allclose(bin_edges, bin_edges_np)
    assessed_ranges_bin_counts, _ = np.histogram(
        distributions[unique_code]["values"], bins=bin_edges
    )
    
    our_values_bin_idx = np.digitize(our_values, bins=bin_edges)

    existing_weighted_bin_counts = np.nan * np.zeros(
        our_values_bin_counts.shape[0]
    )
    for i in range(existing_weighted_bin_counts.shape[0]):
        existing_weighted_bin_counts[i] = weights[
            (our_values_bin_idx == i + 1)
        ].sum()

    if np.equal(j, 0) and np.equal(k, 0):
        np.testing.assert_equal(
            existing_weighted_bin_counts.sum(), our_values_bin_counts.sum()
        )

    unique_code_weights = np.nan * np.zeros(bin_edges.shape[0] + 1)

    # existing_weighted_bin_counts[0] refers to samples outside the
    # assessed range's lower bound. Accordingly, if `our_values` was
    # digitized into a bin idx of zero, it should get a weight of zero.
    unique_code_weights[0] = 0
    # Similarly, if `our_values` was digitized into a bin idx greater
    # than the number of bins then it was outside the assessed range
    # so get a weight of zero.
    unique_code_weights[-1] = 0
    
    for i in range(1, our_values_bin_counts.shape[0] + 1):
        # the histogram idx is one less because digitize gives values in the
        # range bin_edges[0] <= x < bin_edges[1] a digitized index of 1
        histogram_idx = i - 1
        if np.equal(assessed_ranges_bin_counts[histogram_idx], 0):
            unique_code_weights[i] = 0
        elif np.equal(existing_weighted_bin_counts[histogram_idx], 0):
            # other variables force this box to be zero so just fill it with
            # one
            unique_code_weights[i] = 1
        else:
            unique_code_weights[i] = (
                assessed_ranges_bin_counts[histogram_idx]
                / existing_weighted_bin_counts[histogram_idx]
            )
            
    return unique_code_weights, our_values_bin_idx


In [ ]:
weights, gofs, gofs_full = calculate_sample_weights(
    ar_distributions, accepted, niterations=30
)

In [ ]:
gofs_full

In [ ]:
weights

In [ ]:
gofs

In [ ]:
effective_samples = int(np.floor(np.sum(np.minimum(weights, 1))))
effective_samples

In [ ]:
draws = []

drawn_samples = accepted.sample(n=output_ensemble_size, replace=False, weights=weights, random_state=10099)
draws.append((drawn_samples))

#for i in tqdman.tqdm(range(10)):
#    drawn_samples = accepted.sample(n=OUTPUT_ENSEMBLE_SIZE, replace=False, weights=weights, random_state=)
    #     drawn_samples = samples_df.sample(n=10, replace=False, weights=weights)

    #summary_table = get_summary_table(drawn_samples)

    #score, coloured_df = utils.plotting.colour_df(summary_table, transpose=True)

#    draws.append((drawn_samples))

#sorted([v[1] for v in draws])[:10]

In [ ]:
draws[0]

In [ ]:
pl.hist(draws[0]['temperature 1995-2014'])

In [ ]:
target_ecs = scipy.stats.gaussian_kde(samples['ECS'])
prior_ecs = scipy.stats.gaussian_kde(ecs_in)
post1_ecs = scipy.stats.gaussian_kde(ecs_in[valid_temp])
post2_ecs = scipy.stats.gaussian_kde(draws[0]['ECS'])

target_tcr = scipy.stats.gaussian_kde(samples['TCR'])
prior_tcr = scipy.stats.gaussian_kde(tcr_in)
post1_tcr = scipy.stats.gaussian_kde(tcr_in[valid_temp])
post2_tcr = scipy.stats.gaussian_kde(draws[0]['TCR'])

target_temp = scipy.stats.gaussian_kde(samples['temperature 1995-2014'])
prior_temp = scipy.stats.gaussian_kde(temp_in[145:165, :].mean(axis=0))
post1_temp = scipy.stats.gaussian_kde(temp_in[145:165, valid_temp].mean(axis=0))
post2_temp = scipy.stats.gaussian_kde(draws[0]['temperature 1995-2014'])

target_ohc = scipy.stats.gaussian_kde(samples['OHC'])
prior_ohc = scipy.stats.gaussian_kde(ohc_in/1e21)
post1_ohc = scipy.stats.gaussian_kde(ohc_in[valid_temp]/1e21)
post2_ohc = scipy.stats.gaussian_kde(draws[0]['OHC'])

target_aer = scipy.stats.gaussian_kde(samples['ERFaer'])
prior_aer = scipy.stats.gaussian_kde(faer_in)
post1_aer = scipy.stats.gaussian_kde(faer_in[valid_temp])
post2_aer = scipy.stats.gaussian_kde(draws[0]['ERFaer'])

target_aci = scipy.stats.gaussian_kde(samples['ERFaci'])
prior_aci = scipy.stats.gaussian_kde(faci_in)
post1_aci = scipy.stats.gaussian_kde(faci_in[valid_temp])
post2_aci = scipy.stats.gaussian_kde(draws[0]['ERFaci'])

target_ari = scipy.stats.gaussian_kde(samples['ERFari'])
prior_ari = scipy.stats.gaussian_kde(fari_in)
post1_ari = scipy.stats.gaussian_kde(fari_in[valid_temp])
post2_ari = scipy.stats.gaussian_kde(draws[0]['ERFari'])

target_co2 = scipy.stats.gaussian_kde(samples['CO2 concentration'])
prior_co2 = scipy.stats.gaussian_kde(co2_in)
post1_co2 = scipy.stats.gaussian_kde(co2_in[valid_temp])
post2_co2 = scipy.stats.gaussian_kde(draws[0]['CO2 concentration'])

In [ ]:
colors = {
    'prior': '#207F6E',
    'post1': '#684C94',
    'post2': '#EE696B',
    'target': 'black'
}

In [ ]:
fig,ax = pl.subplots(2,3, figsize=(15, 10))
start = 0
stop = 8
ax[0,0].plot(np.linspace(start, stop, 1000), prior_ecs(np.linspace(start, stop, 1000)), color=colors['prior'], label='Prior')
ax[0,0].plot(np.linspace(start, stop, 1000), post1_ecs(np.linspace(start, stop, 1000)), color=colors['post1'], label='Temperature RMSE')
ax[0,0].plot(np.linspace(start, stop, 1000), post2_ecs(np.linspace(start, stop, 1000)), color=colors['post2'], label='All constraints')
ax[0,0].plot(np.linspace(start, stop, 1000), target_ecs(np.linspace(start, stop, 1000)), color=colors['target'], label='Target')
ax[0,0].set_xlim(start, stop)
ax[0,0].set_ylim(0, 0.5)
ax[0,0].set_title('ECS')
ax[0,0].set_yticklabels([])
ax[0,0].set_xlabel('°C')

start = 0
stop = 4
ax[0,1].plot(np.linspace(start, stop, 1000), prior_tcr(np.linspace(start, stop, 1000)), color=colors['prior'], label='Prior')
ax[0,1].plot(np.linspace(start, stop, 1000), post1_tcr(np.linspace(start, stop, 1000)), color=colors['post1'], label='Temperature RMSE')
ax[0,1].plot(np.linspace(start, stop, 1000), post2_tcr(np.linspace(start, stop, 1000)), color=colors['post2'], label='All constraints')
ax[0,1].plot(np.linspace(start, stop, 1000), target_tcr(np.linspace(start, stop, 1000)), color=colors['target'], label='Target')
ax[0,1].set_xlim(start, stop)
ax[0,1].set_ylim(0, 1.2)
ax[0,1].set_title('TCR')
ax[0,1].set_yticklabels([])
ax[0,1].set_xlabel('°C')

start = 0.5
stop = 1.3
ax[0,2].plot(np.linspace(start, stop, 1000), target_temp(np.linspace(start, stop, 1000)), color=colors['target'], label='Target')
ax[0,2].plot(np.linspace(start, stop, 1000), prior_temp(np.linspace(start, stop, 1000)), color=colors['prior'], label='Prior')
ax[0,2].plot(np.linspace(start, stop, 1000), post1_temp(np.linspace(start, stop, 1000)), color=colors['post1'], label='Temperature RMSE')
ax[0,2].plot(np.linspace(start, stop, 1000), post2_temp(np.linspace(start, stop, 1000)), color=colors['post2'], label='All constraints')
ax[0,2].set_xlim(start, stop)
ax[0,2].set_ylim(0, 5)
ax[0,2].set_title('Temperature anomaly')
ax[0,2].set_yticklabels([])
ax[0,2].set_xlabel('°C, 1995-2014 minus 1850-1900')

# start = -2.5
# stop = 0
# ax[1,0].plot(np.linspace(start, stop), target_aer(np.linspace(start, stop)), label='Target distribution')
# ax[1,0].plot(np.linspace(start, stop), prior_aer(np.linspace(start, stop)), label='RMSE < 0.16 K posterior')
# ax[1,0].plot(np.linspace(start, stop), post_aer(np.linspace(start, stop)))
# ax[1,0].set_xlim(start, stop)
# ax[1,0].set_ylim(0, 1.2)

start = -3
stop = 0
ax[1,0].plot(np.linspace(start, stop, 1000), target_aer(np.linspace(start, stop, 1000)), color=colors['target'], label='Target')
ax[1,0].plot(np.linspace(start, stop, 1000), prior_aer(np.linspace(start, stop, 1000)), color=colors['prior'], label='Prior')
ax[1,0].plot(np.linspace(start, stop, 1000), post1_aer(np.linspace(start, stop, 1000)), color=colors['post1'], label='Temperature RMSE')
ax[1,0].plot(np.linspace(start, stop, 1000), post2_aer(np.linspace(start, stop, 1000)), color=colors['post2'], label='All constraints')
ax[1,0].set_xlim(start, stop)
ax[1,0].set_ylim(0, 1.6)
ax[1,0].set_title('Aerosol ERF')
ax[1,0].legend(frameon=False, loc='upper left')
ax[1,0].set_yticklabels([])
ax[1,0].set_xlabel('W m$^{-2}$, 2005-2014 minus 1750')

start = 394
stop = 402
ax[1,1].plot(np.linspace(start, stop, 1000), target_co2(np.linspace(start, stop, 1000)), color=colors['target'], label='Target')
ax[1,1].plot(np.linspace(start, stop, 1000), prior_co2(np.linspace(start, stop, 1000)), color=colors['prior'], label='Prior')
ax[1,1].plot(np.linspace(start, stop, 1000), post1_co2(np.linspace(start, stop, 1000)), color=colors['post1'], label='Temperature RMSE')
ax[1,1].plot(np.linspace(start, stop, 1000), post2_co2(np.linspace(start, stop, 1000)), color=colors['post2'], label='All constraints')
ax[1,1].set_xlim(start, stop)
ax[1,1].set_ylim(0, 1.2)
ax[1,1].set_title('CO$_2$ concentration')
ax[1,1].set_yticklabels([])
ax[1,1].set_xlabel('ppm, 2014')

start = 0
stop = 800
ax[1,2].plot(np.linspace(start, stop), target_ohc(np.linspace(start, stop)), color=colors['target'], label='Target')
ax[1,2].plot(np.linspace(start, stop), prior_ohc(np.linspace(start, stop)), color=colors['prior'], label='Prior')
ax[1,2].plot(np.linspace(start, stop), post1_ohc(np.linspace(start, stop)), color=colors['post1'], label='Temperature RMSE')
ax[1,2].plot(np.linspace(start, stop), post2_ohc(np.linspace(start, stop)), color=colors['post2'], label='All constraints')
ax[1,2].set_xlim(start, stop)
ax[1,2].set_ylim(0, .006)
ax[1,2].set_title('Ocean heat content change')
ax[1,2].set_yticklabels([])
ax[1,2].set_xlabel('ZJ, 2018 minus 1971')
#ax[0,1].hist(samples['TCR'], bins=100, alpha=0.5, label='Target distribution', density=True)
#ax[0,1].hist(tcr_in[valid_temp], bins=100, alpha=0.5, label='RMSE < 0.16 K posterior', density=True);
#ax[0,1].hist(draws[0]['TCR'], bins=100, alpha=0.5, density=True);
#ax[0,0].hist(samples['ECS'], bins=100, alpha=0.5, label='Target distribution', density=True)
#ax[0,0].hist(ecs_in[valid_temp], bins=100, alpha=0.5, label='RMSE < 0.16 K posterior', density=True)

fig.tight_layout()
# pl.savefig('../plots/constraints.png')
# pl.savefig('../plots/constraints.pdf')

In [ ]:
np.percentile(draws[0]['ECS'], (5, 50, 95))

In [ ]:
np.percentile(draws[0]['TCR'], (5, 50, 95))

In [ ]:
pl.scatter(draws[0]['TCR'], draws[0]['ECS'])

In [ ]:
np.percentile(draws[0]['CO2 concentration'], (5, 50, 95))

In [ ]:
np.percentile(draws[0]['temperature 1995-2014'], (5, 50, 95))

In [ ]:
np.percentile(draws[0]['ERFari'], (5, 50, 95))

In [ ]:
np.percentile(draws[0]['ERFaci'], (5, 50, 95))

In [ ]:
np.percentile(draws[0]['ERFaci']+draws[0]['ERFari'], (5, 50, 95))

In [ ]:
np.percentile(draws[0]['OHC'], (5, 50, 95))

In [ ]:
pl.scatter(draws[0]['TCR'], draws[0]['ERFaci']+draws[0]['ERFari'])

In [ ]:
draws[0].index

In [ ]:
df_gmst = pd.read_csv('../../../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

In [ ]:
fig, ax = pl.subplots(figsize=(5, 5))
ax.fill_between(
    np.arange(1850, 2102), 
    np.min(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), axis=1), 
    np.max(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850, 2102), 
    np.percentile(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), 5, axis=1), 
    np.percentile(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), 95, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.fill_between(
    np.arange(1850, 2102), 
    np.percentile(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), 16, axis=1), 
    np.percentile(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), 84, axis=1),
    color='#000000',
    alpha=0.2,
)
ax.plot(
    np.arange(1850, 2102), 
    np.median(temp_in[:,draws[0].index]-np.average(temp_in[:52,draws[0].index], weights=weights_51yr, axis=0), axis=1), 
    color='#000000',
)

ax.plot(np.arange(1850.5, 2021), gmst, color='b', label='Observations')

ax.legend(frameon=False, loc='upper left')

ax.set_xlim(1850,2100)
ax.set_ylim(-1, 5)
ax.set_ylabel('°C relative to 1850-1900')
ax.axhline(0, color='k', ls=":", lw=0.5)
pl.tight_layout()
#pl.title('constrained & reweighted (1001)')
#pl.savefig('../plots/final_reweighted_ssp245.png')

In [ ]:
np.savetxt(f'../../../output/fair-{fair_v}/v{cal_v}/posteriors/{constraint_set}/runids_rmse_reweighted_pass.csv', sorted(draws[0].index))